In [1]:
import pandas as pd
import googlemaps
import json

In [2]:
def get_address(gmaps, latitude, longitude):

    commune = ''
    province = ''
    region = ''
    country = ''

    # Look up an address with reverse geocoding
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

    if len(reverse_geocode_result[0]['address_components'])>=4:

        commune = reverse_geocode_result[0]['address_components'][
            len(reverse_geocode_result[0]['address_components'])-4]['long_name']
        province = reverse_geocode_result[0]['address_components'][
            len(reverse_geocode_result[0]['address_components'])-3]['long_name']
        region = reverse_geocode_result[0]['address_components'][
            len(reverse_geocode_result[0]['address_components'])-2]['long_name']
        country = reverse_geocode_result[0]['address_components'][
            len(reverse_geocode_result[0]['address_components'])-1]['long_name']

    return commune, province, region, country

In [3]:
with open('api_key.json') as json_file:
        data = json.load(json_file)
        api_key = data['api_key'][0]


In [4]:
stations_df = pd.read_csv('stations.csv')

In [6]:
# Connect to google maps api by using api key
g = googlemaps.Client(key=api_key)

In [7]:
# Look up an addresses with reverse geocoding
unpacked_tuple_df = pd.DataFrame(
    stations_df.apply(
        lambda x: get_address(g, x['latitud'], x['longitud']),
        axis=1).tolist(), columns=['comuna', 'provincia', 'region', 'pais'], index=stations_df.index)
stations_df = pd.concat([stations_df, unpacked_tuple_df], axis=1)
stations_df


,codigo_estacion,nombre,latitud,longitud,comuna,provincia,region,pais
0,01000005,Visviri,-17.5950,-69.4831,General Lagos,Parinacota Province,Arica y Parinacota,Chile
1,01001005,Caquena,-18.0542,-69.2017,Putre,Parinacota Province,Arica y Parinacota,Chile
2,01010007,Chungara Reten,-18.2808,-69.1383,Putre,Parinacota,Arica y Parinacota,Chile
3,01010010,Chungara Ajata,-18.2353,-69.1833,Putre,Parinacota,Arica y Parinacota,Chile
4,01020002,Rio Desaguadero Cotacotani,-18.1936,-69.2458,Putre,Parinacota Province,Arica y Parinacota,Chile
...,...,...,...,...,...,...,...,...
968,VEM00080437,El Vigia,8.6330,-71.6500,Alberto Adriani,Mérida,Venezuela,5145
969,VEM00080440,Barinas,8.6200,-70.2210,Barinas,Barinas,Venezuela,5201
970,VEM00080447,San Antonio Del Tachira,7.8410,-72.4400,Bolívar,Táchira,Venezuela,5007
971,VEM00080448,Guasdualito,7.2110,-70.7560,Paez,Apure,Venezuela,5063


In [8]:
stations_df.to_csv('stations_locations.csv', sep=';', index=False)